In [1]:
import keras.backend as K

Using TensorFlow backend.


In [0]:
import tensorflow as tf

In [0]:
import numpy
import skimage.draw

In [0]:
tf.enable_eager_execution()

version 4

In [0]:
def mrcnn_mask_loss_graph(target_masks, target_class_ids, pred_masks):
    """Mask binary cross-entropy loss for the masks head.
    target_masks: [batch, num_rois, height, width].
        A float32 tensor of values 0 or 1. Uses zero padding to fill array.
    target_class_ids: [batch, num_rois]. Integer class IDs. Zero padded.
    pred_masks: [batch, proposals, height, width, num_classes] float32 tensor
                with values from 0 to 1.
    """
    # Reshape for simplicity. Merge first two dimensions into one.
    target_class_ids = K.reshape(target_class_ids, (-1,))
    mask_shape = tf.shape(target_masks)
    target_masks = K.reshape(target_masks, (-1, mask_shape[2], mask_shape[3]))
    pred_shape = tf.shape(pred_masks)
    pred_masks = K.reshape(pred_masks,
                           (-1, pred_shape[2], pred_shape[3], pred_shape[4]))
    # Permute predicted masks to [N, num_classes, height, width]
    pred_masks = tf.transpose(pred_masks, [0, 3, 1, 2])

    # Only positive ROIs contribute to the loss. And only
    # the class specific mask of each ROI.
    positive_ix = tf.where(target_class_ids > 0)[:, 0]
    positive_class_ids = tf.cast(
        tf.gather(target_class_ids, positive_ix), tf.int64)
    indices = tf.stack([positive_ix, positive_class_ids], axis=1)

    # Gather the masks (predicted and true) that contribute to loss
    y_true = tf.gather(target_masks, positive_ix)
    y_pred = tf.gather_nd(pred_masks, indices)
    
    m_zer = tf.constant(0.0, dtype=tf.float32)
    m_one = tf.constant(1.0, dtype=tf.float32)
    m_sev = tf.constant(0.7, dtype=tf.float32)
    outside = tf.multiply(tf.subtract(m_one, y_true), y_pred)
    inside = tf.multiply(y_true, y_pred)
    inside_ratio = tf.subtract(m_one, tf.minimum(tf.divide(tf.reduce_sum(inside), tf.multiply(tf.reduce_sum(y_true), m_sev)), m_one))

    #sum reduce the inside
    #compare against .7 * sum reduce actual GT
    #multiply against BCE
    
    
    # Compute binary cross entropy. If no positive ROIs, then return 0.
    # shape: [batch, roi, num_classes]
    loss = K.switch(tf.size(y_true) > 0,
                    K.binary_crossentropy(target=tf.zeros_like(outside), output=outside),
                    tf.constant(0.0))
    loss = K.mean(loss)
    
    loss_in = K.switch(tf.size(y_true) > 0,
                    K.binary_crossentropy(target=y_pred, output=inside),
                    tf.constant(0.0))
    loss_in = K.mean(loss_in)

#     print(loss)
#     print(outside)
#     lolz = tf.multiply(tf.cast(factor, tf.float32), loss)
#     print(lolz)
    return tf.add(loss, tf.multiply(inside_ratio, loss_in))

In [0]:
img = numpy.zeros((20, 20), dtype=numpy.uint8)
rr, cc = skimage.draw.circle(16, 9, 4)
img[rr, cc] = 1
rr, cc = skimage.draw.circle(9, 9, 4)
img[rr, cc] = 2
img

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1,

In [32]:
gt = numpy.zeros((1, 1, 20, 20), dtype=numpy.float32)
rr, cc = skimage.draw.circle(16, 9, 4)
gt[0, 0, rr, cc] = 1
c_ids = numpy.zeros((1, 1), dtype=numpy.float32)
c_ids[0, 0] = 1
pred = numpy.zeros((1, 1, 20, 20, 2), dtype=numpy.float32)
for g in range(1, 11):
  print('confidence %f' % (g / 10))
  for i in range(9, 17):

    rr, cc = skimage.draw.circle(i, 9, 4)
    pred[0, 0, rr, cc, 1] = g / 10
    #print(pred[0, 0, :, :, 1])
    print(mrcnn_mask_loss_graph(gt, c_ids, pred))
    pred[0, 0, :, :, 1] = 0

confidence 0.100000
tf.Tensor(0.19318178, shape=(), dtype=float32)
tf.Tensor(0.17315778, shape=(), dtype=float32)
tf.Tensor(0.15364437, shape=(), dtype=float32)
tf.Tensor(0.12718341, shape=(), dtype=float32)
tf.Tensor(0.10172323, shape=(), dtype=float32)
tf.Tensor(0.077263854, shape=(), dtype=float32)
tf.Tensor(0.053805266, shape=(), dtype=float32)
tf.Tensor(0.031347465, shape=(), dtype=float32)
confidence 0.200000
tf.Tensor(0.38776094, shape=(), dtype=float32)
tf.Tensor(0.3404992, shape=(), dtype=float32)
tf.Tensor(0.29539868, shape=(), dtype=float32)
tf.Tensor(0.23588891, shape=(), dtype=float32)
tf.Tensor(0.18061529, shape=(), dtype=float32)
tf.Tensor(0.12957776, shape=(), dtype=float32)
tf.Tensor(0.08277636, shape=(), dtype=float32)
tf.Tensor(0.04021107, shape=(), dtype=float32)
confidence 0.300000
tf.Tensor(0.58411175, shape=(), dtype=float32)
tf.Tensor(0.50345683, shape=(), dtype=float32)
tf.Tensor(0.4278311, shape=(), dtype=float32)
tf.Tensor(0.33040428, shape=(), dtype=float32)

version 0

In [0]:
def mrcnn_mask_loss_graph(target_masks, target_class_ids, pred_masks):
    """Mask binary cross-entropy loss for the masks head.
    target_masks: [batch, num_rois, height, width].
        A float32 tensor of values 0 or 1. Uses zero padding to fill array.
    target_class_ids: [batch, num_rois]. Integer class IDs. Zero padded.
    pred_masks: [batch, proposals, height, width, num_classes] float32 tensor
                with values from 0 to 1.
    """
    # Reshape for simplicity. Merge first two dimensions into one.
    target_class_ids = K.reshape(target_class_ids, (-1,))
    mask_shape = tf.shape(target_masks)
    target_masks = K.reshape(target_masks, (-1, mask_shape[2], mask_shape[3]))
    pred_shape = tf.shape(pred_masks)
    pred_masks = K.reshape(pred_masks,
                           (-1, pred_shape[2], pred_shape[3], pred_shape[4]))
    # Permute predicted masks to [N, num_classes, height, width]
    pred_masks = tf.transpose(pred_masks, [0, 3, 1, 2])

    # Only positive ROIs contribute to the loss. And only
    # the class specific mask of each ROI.
    positive_ix = tf.where(target_class_ids > 0)[:, 0]
    positive_class_ids = tf.cast(
        tf.gather(target_class_ids, positive_ix), tf.int64)
    indices = tf.stack([positive_ix, positive_class_ids], axis=1)

    # Gather the masks (predicted and true) that contribute to loss
    y_true = tf.gather(target_masks, positive_ix)
    y_pred = tf.gather_nd(pred_masks, indices)
    
    m_zer = tf.constant(0.0, dtype=tf.float32)
    m_one = tf.constant(1.0, dtype=tf.float32)
    m_sev = tf.constant(0.7, dtype=tf.float32)
    outside = tf.count_nonzero(tf.multiply(tf.subtract(m_one, y_true), y_pred), dtype=tf.float32)
    inside = tf.divide(tf.count_nonzero(tf.multiply(y_true, y_pred), dtype=tf.float32), tf.count_nonzero(y_true, dtype=tf.float32))
    factor = tf.cond(tf.less(inside, m_sev), lambda: tf.subtract(m_one, tf.divide(inside, m_sev)), lambda: m_zer)

    # Compute binary cross entropy. If no positive ROIs, then return 0.
    # shape: [batch, roi, num_classes]
    loss = K.switch(tf.size(y_true) > 0,
                    K.binary_crossentropy(target=y_true, output=y_pred),
                    tf.constant(0.0))
    loss = K.mean(loss)
#     print(loss)
#     print(outside)
#     lolz = tf.multiply(tf.cast(factor, tf.float32), loss)
#     print(lolz)
    return tf.add(outside, tf.multiply(factor, loss))

https://stackoverflow.com/questions/44657388/how-to-replace-certain-values-in-tensorflow-tensor-with-the-values-of-the-other?rq=1

to calculate area masked but not in ground truth, multiply zeroed GT with predicted and count non zeros

if that is zero, then compute percentage P, how much of the ground truth is predicted.

substract by 100
if 100-P > 30, then apply binary cross entropy?

WIP TODO

https://stackoverflow.com/questions/35833011/how-to-add-if-condition-in-a-tensorflow-graph

In [0]:
gt = numpy.zeros((1, 1, 28, 28), dtype=numpy.float32)
c_ids = numpy.zeros((1, 1), dtype=numpy.float32)
c_ids[0, 0] = 1
pred = numpy.zeros((1, 1, 28, 28, 2), dtype=numpy.float32)
pred[0, 0, :, :, 0] = 0
pred[0, 0, :, :, 1] = 1

In [0]:
lmao = mrcnn_mask_loss_graph(gt, c_ids, pred)

In [0]:
type(lmao)

tensorflow.python.framework.ops.EagerTensor

In [0]:
lmao

<tf.Tensor: id=261, shape=(), dtype=float32, numpy=15.942384>

In [0]:
#tf.print()

In [0]:
import sys

In [0]:
tf.enable_eager_execution()

@tf.contrib.eager.defun
def f():
    tensor = tf.range(10)
    tf.print(tensor, output_stream=sys.stderr)
    return tensor

range_tensor = f()

In [0]:
range_tensor

<tf.Tensor: id=9, shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [0]:
tf.enable_eager_execution()
tensor = tf.range(10)
tf.print(tensor, output_stream=sys.stderr)

In [0]:
sess = K.get_session()
with sess.as_default():
  tensor = tf.range(10)
  print_op = tf.print(tensor)
  with tf.control_dependencies([print_op]):
    out = tf.add(tensor, tensor)
  sess.run(out)

In [0]:
# sess = tf.Session()
# with sess.as_default():
#     lmao = mrcnn_mask_loss_graph(gt, c_ids, pred)
#     print_op = tf.print(tensor)
#     with tf.control_dependencies([print_op]):
#       out = tf.add(tensor, tensor)
#     sess.run(out)

version 1

In [0]:
def mrcnn_mask_loss_graph(target_masks, target_class_ids, pred_masks):
    """Mask binary cross-entropy loss for the masks head.

    target_masks: [batch, num_rois, height, width].
        A float32 tensor of values 0 or 1. Uses zero padding to fill array.
    target_class_ids: [batch, num_rois]. Integer class IDs. Zero padded.
    pred_masks: [batch, proposals, height, width, num_classes] float32 tensor
                with values from 0 to 1.
    """
    # Reshape for simplicity. Merge first two dimensions into one.
    target_class_ids = K.reshape(target_class_ids, (-1,))
    mask_shape = tf.shape(target_masks)
    target_masks = K.reshape(target_masks, (-1, mask_shape[2], mask_shape[3]))
    pred_shape = tf.shape(pred_masks)
    pred_masks = K.reshape(pred_masks,
                           (-1, pred_shape[2], pred_shape[3], pred_shape[4]))
    # Permute predicted masks to [N, num_classes, height, width]
    pred_masks = tf.transpose(pred_masks, [0, 3, 1, 2])

    # Only positive ROIs contribute to the loss. And only
    # the class specific mask of each ROI.
    positive_ix = tf.where(target_class_ids > 0)[:, 0]
    positive_class_ids = tf.cast(
        tf.gather(target_class_ids, positive_ix), tf.int64)
    indices = tf.stack([positive_ix, positive_class_ids], axis=1)

	#################
	# CUSTOM LOSS CODE BY DEREK ZHANG
	# BCE = Binary Cross Entropy
	# return (BCE outside GT mask) + (BCE inside GT mask) * ((percent GT mask predicted < 70%) ? (percent GT mask predicted / 70%) : 0)
	#################

	m_zer = tf.constant(0.0, dtype=tf.float32)
    m_one = tf.constant(1.0, dtype=tf.float32)
    m_sev = tf.constant(0.7, dtype=tf.float32)
    outside = tf.multiply(tf.subtract(m_one, y_true), y_pred)
    inside = tf.multiply(y_true, y_pred)
    inside_ratio = tf.divide(tf.count_nonzero(inside, dtype=tf.float32), tf.count_nonzero(y_true, dtype=tf.float32))
    factor = tf.cond(tf.less(inside_ratio, m_sev), lambda: tf.subtract(m_one, tf.divide(inside_ratio, m_sev)), lambda: m_zer)

    # Compute binary cross entropy. If no positive ROIs, then return 0.
    # shape: [batch, roi, num_classes]
    loss = K.switch(tf.size(y_true) > 0,
                    K.binary_crossentropy(target=tf.zeros_like(outside), output=outside),
                    tf.constant(0.0))
    loss = K.mean(loss)

    loss_in = K.switch(tf.size(y_true) > 0,
                    K.binary_crossentropy(target=y_pred, output=inside),
                    tf.constant(0.0))
    loss_in = K.mean(loss_in)

    return tf.add(loss, tf.multiply(factor, loss_in))

failed version 3

In [0]:
def not_good_enough_mask(inside_ratio, y_true, y_pred, inside):

    m_one = tf.constant(1.0, dtype=tf.float32)
    m_sev = tf.constant(0.7, dtype=tf.float32)
    factor = tf.subtract(m_one, tf.divide(inside_ratio, m_sev))
    loss_in = K.switch(tf.size(y_true) > 0,
                    K.binary_crossentropy(target=y_pred, output=inside),
                    tf.constant(0.0))
    loss_in = K.mean(loss_in)
    return tf.multiply(factor, loss_in)
  #min(x/0.7, 1)

In [0]:
def good_enough_mask(inside):
  
  m_one = tf.constant(1.0, dtype=tf.float32)
  
  return tf.subtract(m_one, K.mean(inside))

In [0]:
def mrcnn_mask_loss_graph(target_masks, target_class_ids, pred_masks):
    """Mask binary cross-entropy loss for the masks head.

    target_masks: [batch, num_rois, height, width].
        A float32 tensor of values 0 or 1. Uses zero padding to fill array.
    target_class_ids: [batch, num_rois]. Integer class IDs. Zero padded.
    pred_masks: [batch, proposals, height, width, num_classes] float32 tensor
                with values from 0 to 1.
    """
    # Reshape for simplicity. Merge first two dimensions into one.
    target_class_ids = K.reshape(target_class_ids, (-1,))
    mask_shape = tf.shape(target_masks)
    target_masks = K.reshape(target_masks, (-1, mask_shape[2], mask_shape[3]))
    pred_shape = tf.shape(pred_masks)
    pred_masks = K.reshape(pred_masks,
                           (-1, pred_shape[2], pred_shape[3], pred_shape[4]))
    # Permute predicted masks to [N, num_classes, height, width]
    pred_masks = tf.transpose(pred_masks, [0, 3, 1, 2])

    # Only positive ROIs contribute to the loss. And only
    # the class specific mask of each ROI.
    positive_ix = tf.where(target_class_ids > 0)[:, 0]
    positive_class_ids = tf.cast(
        tf.gather(target_class_ids, positive_ix), tf.int64)
    indices = tf.stack([positive_ix, positive_class_ids], axis=1)
    
    # Gather the masks (predicted and true) that contribute to loss
    y_true = tf.gather(target_masks, positive_ix)
    y_pred = tf.gather_nd(pred_masks, indices)

    #################
    # CUSTOM LOSS CODE BY DEREK ZHANG
    # BCE = Binary Cross Entropy
    # return (BCE outside GT mask) + (BCE inside GT mask) * ((percent GT mask predicted < 70%) ? (percent GT mask predicted / 70%) : 0)
    #################

    #m_zer = tf.constant(0.0, dtype=tf.float32)
    m_one = tf.constant(1.0, dtype=tf.float32)
    m_sev = tf.constant(0.7, dtype=tf.float32)
    outside = tf.multiply(tf.subtract(m_one, y_true), y_pred)
    inside = tf.multiply(y_true, y_pred)
    inside_ratio = tf.divide(tf.count_nonzero(inside, dtype=tf.float32), tf.count_nonzero(y_true, dtype=tf.float32))
    inside_stuff = tf.cond(tf.less(inside_ratio, m_sev), not_good_enough_mask(inside_ratio, y_true, y_pred, inside), lambda: good_enough_mask(inside))

    # Compute binary cross entropy. If no positive ROIs, then return 0.
    # shape: [batch, roi, num_classes]
    loss = K.switch(tf.size(y_true) > 0,
                    K.binary_crossentropy(target=tf.zeros_like(outside), output=outside),
                    tf.constant(0.0))
    loss = K.mean(loss)

    return tf.add(loss, inside_stuff)

failed version 2

In [0]:
def mrcnn_mask_loss_graph(target_masks, target_class_ids, pred_masks):
    """Mask binary cross-entropy loss for the masks head.
    target_masks: [batch, num_rois, height, width].
        A float32 tensor of values 0 or 1. Uses zero padding to fill array.
    target_class_ids: [batch, num_rois]. Integer class IDs. Zero padded.
    pred_masks: [batch, proposals, height, width, num_classes] float32 tensor
                with values from 0 to 1.
    """
    # Reshape for simplicity. Merge first two dimensions into one.
    target_class_ids = K.reshape(target_class_ids, (-1,))
    mask_shape = tf.shape(target_masks)
    target_masks = K.reshape(target_masks, (-1, mask_shape[2], mask_shape[3]))
    pred_shape = tf.shape(pred_masks)
    pred_masks = K.reshape(pred_masks,
                           (-1, pred_shape[2], pred_shape[3], pred_shape[4]))
    # Permute predicted masks to [N, num_classes, height, width]
    pred_masks = tf.transpose(pred_masks, [0, 3, 1, 2])

    # Only positive ROIs contribute to the loss. And only
    # the class specific mask of each ROI.
    positive_ix = tf.where(target_class_ids > 0)[:, 0]
    positive_class_ids = tf.cast(
        tf.gather(target_class_ids, positive_ix), tf.int64)
    indices = tf.stack([positive_ix, positive_class_ids], axis=1)

    # Gather the masks (predicted and true) that contribute to loss
    y_true = tf.gather(target_masks, positive_ix)
    y_pred = tf.gather_nd(pred_masks, indices)
    
    m_zer = tf.constant(0.0, dtype=tf.float32)
    m_one = tf.constant(1.0, dtype=tf.float32)
    m_sev = tf.constant(0.7, dtype=tf.float32)
    outside = tf.multiply(tf.subtract(m_one, y_true), y_pred)
    inside = tf.multiply(y_true, y_pred)
    inside_ratio = tf.divide(tf.count_nonzero(inside, dtype=tf.float32), tf.count_nonzero(y_true, dtype=tf.float32))

    output = tf.cond(tf.less(inside_ratio, m_sev), not_good_enough_mask(m_one, inside_ratio, m_sev, inside), good_enough_mask())

    
    
    return tf.add(loss, tf.multiply(factor, loss_in))